In [1]:
%load_ext autoreload
%autoreload 2

In [30]:
import multiprocessing
import os
import pathlib
import xarray as xr

from tqdm.notebook import tqdm
from crims2s.util import fix_dataset_dims

In [22]:
INPUT_TRAIN = '***BASEDIR***training-input/0.3.0/netcdf'
OUTPUT_TRAIN = '***BASEDIR***processed/training-input/'
OBSERVATIONS = '***BASEDIR***training-output-reference/'
BENCHNMARK = '***BASEDIR***training-output-benchmark/'
CENTER = 'ecmwf'
FIELD = 't'

In [23]:
input_path = pathlib.Path(INPUT_TRAIN)
output_path = pathlib.Path(OUTPUT_TRAIN)

In [24]:
output_path.mkdir(parents=True, exist_ok=True)

In [25]:
input_files = sorted([f for f in input_path.iterdir() if CENTER in f.stem and f'-{FIELD}-' in f.stem])

In [26]:
input_files[:2]

[PosixPath('***BASEDIR***training-input/0.3.0/netcdf/ecmwf-hindcast-t-20200102.nc'),
 PosixPath('***BASEDIR***training-input/0.3.0/netcdf/ecmwf-hindcast-t-20200109.nc')]

In [ ]:
def do_one_file(f):
    new_file = output_path / f.name
    
    if new_file.isfile():
        """Ignore file if it already exists and file size is ok."""
        stream_len = int(stream.headers['Content-length'])
        local_len = os.path.getsize(download_path)

        if stream_len == local_len:               
            return download_path    
    
    
    print(new_file)
    d = xr.open_dataset(f, chunks={'plev': 1, 'forecast_time': 1})
    d.to_netcdf(new_file, mode='w')

In [ ]:
with multiprocessing.Pool() as pool:
    for _ in tqdm(pool.imap(download_one_dataset_file, dataset_files), total=len(dataset_files)):
        


In [37]:
for f in input_files[:1]:
    new_file = output_path / f.name
    datestring = f.stem.split('-')[-1]
    
    print(new_file)
    d = xr.open_dataset(f)
    d = fix_dataset_dims(d)
    
    for level in [1000, 925, 850, 700, 500, 300, 200, 100, 50, 10]:
        new_filename = f'{CENTER}-hindcast-{FIELD}-{level}-{datestring}.nc'
        new_path = output_path / new_filename
        subset = d.sel(plev=[level])
        print(new_path)
        
        subset.to_netcdf(new_path, mode='w')
    

***BASEDIR***processed/training-input/ecmwf-hindcast-t-20200102.nc
***BASEDIR***processed/training-input/ecmwf-hindcast-t-1000-20200102.nc
***BASEDIR***processed/training-input/ecmwf-hindcast-t-925-20200102.nc
***BASEDIR***processed/training-input/ecmwf-hindcast-t-850-20200102.nc
***BASEDIR***processed/training-input/ecmwf-hindcast-t-700-20200102.nc
***BASEDIR***processed/training-input/ecmwf-hindcast-t-500-20200102.nc
***BASEDIR***processed/training-input/ecmwf-hindcast-t-300-20200102.nc
***BASEDIR***processed/training-input/ecmwf-hindcast-t-200-20200102.nc
***BASEDIR***processed/training-input/ecmwf-hindcast-t-100-20200102.nc
***BASEDIR***processed/training-input/ecmwf-hindcast-t-50-20200102.nc
***BASEDIR***processed/training-input/ecmwf-hindcast-t-10-20200102.nc


In [34]:
output_files = sorted([f for f in output_path.iterdir() if '20200102' in f.stem])

In [39]:
xr.open_dataset(output_files[0])

<xarray.Dataset>
Dimensions:             (forecast_dayofyear: 1, forecast_year: 20, latitude: 121, lead_time: 47, longitude: 240, plev: 1, realization: 11)
Coordinates:
  * realization         (realization) int64 0 1 2 3 4 5 6 7 8 9 10
    forecast_time       (forecast_year) datetime64[ns] ...
  * lead_time           (lead_time) timedelta64[ns] 0 days 1 days ... 46 days
  * plev                (plev) float64 10.0
  * latitude            (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude           (longitude) float64 0.0 1.5 3.0 ... 355.5 357.0 358.5
    valid_time          (forecast_year, lead_time) datetime64[ns] ...
  * forecast_dayofyear  (forecast_dayofyear) int64 2
  * forecast_year       (forecast_year) int64 2000 2001 2002 ... 2017 2018 2019
Data variables:
    t                   (forecast_year, forecast_dayofyear, plev, lead_time, realization, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-05-10T16:38 GRIB to CDM+CF via cfgrib-0.9.9...

In [40]:
xr.open_mfdataset(output_files)

<xarray.Dataset>
Dimensions:             (forecast_dayofyear: 1, forecast_year: 20, latitude: 121, lead_time: 47, longitude: 240, plev: 10, realization: 11)
Coordinates:
  * realization         (realization) int64 0 1 2 3 4 5 6 7 8 9 10
    forecast_time       (forecast_year) datetime64[ns] dask.array<chunksize=(20,), meta=np.ndarray>
  * lead_time           (lead_time) timedelta64[ns] 0 days 1 days ... 46 days
  * plev                (plev) float64 10.0 50.0 100.0 ... 850.0 925.0 1e+03
  * latitude            (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude           (longitude) float64 0.0 1.5 3.0 ... 355.5 357.0 358.5
    valid_time          (forecast_year, lead_time) datetime64[ns] dask.array<chunksize=(20, 47), meta=np.ndarray>
  * forecast_dayofyear  (forecast_dayofyear) int64 2
  * forecast_year       (forecast_year) int64 2000 2001 2002 ... 2017 2018 2019
Data variables:
    t                   (forecast_year, forecast_dayofyear, plev, lead_time, realization, latitude, longitude) float32 dask.array<chunksize=(20, 1, 1, 47, 11, 121, 240), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-05-10T16:38 GRIB to CDM+CF via cfgrib-0.9.9...

In [ ]:
input_files